In [21]:
import pandas as pd
import numpy as np
import spacy
from gensim.models import KeyedVectors
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import pickle

In [13]:
dados_treino = pd.read_csv(r"..\06-word2vec-interpretação-da-linguagem-humana-com-word-embedding\database-train-project-07.csv")
dados_teste = pd.read_csv(r"..\06-word2vec-interpretação-da-linguagem-humana-com-word-embedding\database-test-project-07.csv")
display(dados_treino.sample(3), dados_teste.sample(3))

,title,text,date,category,subcategory,link
64047,Negociação entre Odebrecht e MPF na Lava Jato ...,A negociação entre a Odebrecht e o Ministério ...,2016-08-26,colunas,monicabergamo,http://www1.folha.uol.com.br/colunas/monicaber...
38754,Irã cancela conferência em Londres sobre contr...,O Irã cancelou uma conferência em Londres onde...,2016-01-30,mercado,NaN,http://www1.folha.uol.com.br/mercado/2016/01/1...
83780,De volta à zona de rebaixamento,Dilma Rousseff vai vetar a contrarreforma da P...,2015-06-18,colunas,viniciustorres,http://www1.folha.uol.com.br/colunas/viniciust...


,title,text,date,category,subcategory,link
10868,Profissional precisa saber se renovar e ir em ...,A cena é bem comum: o chefe convoca uma reuniã...,2017-07-05,colunas,americo-jose,http://www1.folha.uol.com.br/colunas/americo-j...
13162,Neymar é dúvida para último amistoso antes da ...,O atacante Neymar deverá ser poupado no amisto...,2015-02-06,esporte,NaN,http://www1.folha.uol.com.br/esporte/2015/06/1...
996,Morre aos 88 anos o ex-presidente do Uruguai J...,"Morreu nesta segunda-feira (24), aos 88 anos, ...",2016-10-24,mundo,NaN,http://www1.folha.uol.com.br/mundo/2016/10/182...


In [7]:
w2v_modelo_cbow = KeyedVectors.load_word2vec_format(r"modelo_cbow.txt")
w2v_modelo_sg = KeyedVectors.load_word2vec_format(r"modelo_skipgram.txt")

In [4]:
nlp = spacy.load("pt_core_news_sm", disable=["parser", "ner", "tagger", "textcat"])

def tokenizador(texto):
    doc = nlp(texto)
    tokens_validos = []
    for token in doc:
        eh_valido = not token.is_stop and token.is_alpha
        if eh_valido:
            tokens_validos.append(token.text.lower())
    return tokens_validos

texto = "O Palmeiras perdeu 1 semi final da Libertadores para o Boca Juniors em um empate por 1x1!"
tokens = tokenizador(texto)
tokens

['palmeiras', 'perdeu', 'semi', 'libertadores', 'boca', 'juniors', 'empate']

In [9]:
def combinacao_de_vetores_por_soma(palavras, modelo):
    vetor_resultante = np.zeros(300)
    for pn in palavras:
        try:
            vetor_resultante += modelo.get_vector(pn)
        except KeyError:
            pass            
    return vetor_resultante

vetor_texto = combinacao_de_vetores_por_soma(tokens, w2v_modelo_cbow)
print(len(vetor_texto))
print(vetor_texto)

300
[ 0.7801878   4.29842284  0.73926125 -0.59416728 -0.49637394  1.7796951
  2.40363423 -0.0380234  -2.90397643 -1.99402136  2.54409689  0.89306743
 -3.59675396  2.25159883  0.27661065 -1.17617911  0.16990768 -0.35642757
  1.14840138 -0.11481817 -0.22703961 -0.01138543 -0.95771215 -1.55581608
  6.22016279 -2.04617444 -0.01446276 -3.50416368 -1.97179233  0.47403601
  0.46386492 -3.73994265 -1.13137485 -1.20119155 -0.13847727  3.88534208
 -0.64474485 -1.81743231  0.07957454 -2.47289099 -0.3603785  -1.06822372
  0.21558981 -0.22324412 -0.02779173 -0.88661775  0.00894104 -0.76671262
  2.31741935  3.11644012  1.1674245  -2.05426128 -2.6579865   1.85141905
 -3.20050356  0.26815718 -2.63129355  0.75755293  1.27484564 -0.55828496
 -0.20795769  5.79226662  1.44829729 -0.25450409 -0.76137707 -0.79803135
 -2.15402102  0.8003252   1.12725202 -2.47211276 -2.97405961  1.55036492
 -1.67372388  0.40550135 -1.55049349  0.43695844 -0.88900382  0.15613291
 -0.03844961  4.52814457 -0.33301951  2.7577082 

In [10]:
def matriz_vetores(textos, modelo):
    x = len(textos)
    y = 300
    matriz = np.zeros((x,y))

    for i in range(x):
        titulo_tokenizado = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_de_vetores_por_soma(titulo_tokenizado, modelo)

    return matriz

In [14]:
matriz_vetores_treino_cbow = matriz_vetores(dados_treino["title"], w2v_modelo_cbow)
matriz_vetores_teste_cbow = matriz_vetores(dados_teste["title"], w2v_modelo_cbow)
print(matriz_vetores_treino_cbow.shape)
print(matriz_vetores_teste_cbow.shape)

(90000, 300)
(20513, 300)


In [19]:
matriz_vetores_treino_sg = matriz_vetores(dados_treino["title"], w2v_modelo_sg)
matriz_vetores_teste_sg = matriz_vetores(dados_teste["title"], w2v_modelo_sg)
print(matriz_vetores_treino_sg.shape)
print(matriz_vetores_teste_sg.shape)

(90000, 300)
(20513, 300)


In [16]:
def classificador(modelo, x_treino, y_treino, x_teste, y_teste):
    RL = LogisticRegression(max_iter=800)
    RL.fit(x_treino, y_treino)
    categorias = RL.predict(x_teste)
    resultados = classification_report(y_teste, categorias)
    print(resultados)
    return RL

In [17]:
RL_cbow = classificador(modelo= w2v_modelo_cbow,
                        x_treino= matriz_vetores_treino_cbow,
                        y_treino= dados_treino["category"],
                        x_teste= matriz_vetores_teste_cbow,
                        y_teste= dados_teste["category"])

              precision    recall  f1-score   support

     colunas       0.80      0.71      0.75      6103
   cotidiano       0.64      0.81      0.71      1698
     esporte       0.93      0.87      0.89      4663
   ilustrada       0.13      0.85      0.23       131
     mercado       0.84      0.78      0.81      5867
       mundo       0.74      0.83      0.78      2051

    accuracy                           0.79     20513
   macro avg       0.68      0.81      0.70     20513
weighted avg       0.82      0.79      0.80     20513



In [20]:
RL_sg = classificador(modelo= w2v_modelo_sg,
                        x_treino= matriz_vetores_treino_sg,
                        y_treino= dados_treino["category"],
                        x_teste= matriz_vetores_teste_sg,
                        y_teste= dados_teste["category"])

              precision    recall  f1-score   support

     colunas       0.81      0.71      0.76      6103
   cotidiano       0.64      0.80      0.71      1698
     esporte       0.93      0.87      0.90      4663
   ilustrada       0.14      0.85      0.24       131
     mercado       0.84      0.79      0.81      5867
       mundo       0.75      0.84      0.79      2051

    accuracy                           0.79     20513
   macro avg       0.68      0.81      0.70     20513
weighted avg       0.82      0.79      0.80     20513



In [22]:
with open("rl_cbow.pkl", "wb") as f:
    pickle.dump(RL_cbow, f)

In [23]:
with open("rl_sg.pkl", "wb") as f:
    pickle.dump(RL_sg, f)